In [127]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
import sys
sys.path.append('/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [129]:
import pylab
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.signal as scsig
import spectrum as sp
import signale
import cPickle as pkl
import time
import statsmodels.api as sm
import colormaps as cm
import seaborn as sns
import signale.tools as tools
import os
import trajectory
from sklearn.cluster import KMeans
from matplotlib.mlab import specgram
import matplotlib.mlab as mlab
from sklearn import linear_model, datasets

In [130]:
def whitenARMA(sig,AR=2,MA=0):
        arma = sm.tsa.ARMA(sig, (AR,MA)).fit(disp=0)
        print 'ARMA parameters calculated for order(%s,%s)' %(AR,MA)
        return arma.resid
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def nvt_loader(filename):
    """ 
    Memory map the Neuralynx .nvt format
    Fields
    -------
    swstx
    swid
    sw_data_size
    qTimeStamps       Cheetah timestamp for this record. This value is in microseconds.
    dwpoints          Points with the color bitfield values for this record.This is a 400
                      element array.  See Video Tracker in reference. 
    sncrc
    dnextracted_x     Extracted X location of the target being tracked. 
    dnextracted_y     Extracted Y location of the target being tracked.
    dnextracted_angle The calculated head angle in degrees clockwise from the positive Y
                      axis. Zero will be assigned if angle tracking is disabled.
    dnTargets         Colored targets using the samebitfield format used by the dwPoints array.
                      Instead of transitions, the bitfield indicates the colors that make up 
                      each particular target and the center point of that target.  This is a 50
                      element array sorted by size from largest (index 0) to smallest(index 49).
                      A target value of 0 means that no target is present in thatindex location.
                      See Video Tracker Bitfield Information in reference. 
    
    Reference:
    ----------
        http://neuralynx.com/software/NeuralynxDataFileFormats.pdf
    """ 
    nev_dtype = np.dtype([
        ('swstx'              , '<i2'),
        ('swid'               , '<i2'),
        ('sw_data_size'       , '<i2'),
        ('qTimeStamps'        , '<u8'),
        ('dwPoints'           , '<u4',(400,)),
        ('sncrc'              , '<i2'),
        ('dnextracted_x'      , '<i4'),
        ('dnextracted_y'      , '<i4'),
        ('dnextracted_angle'  , '<i4'),
        ('dnTargets'          , '<i4',(50,)),
    ])
    return np.memmap(filename, dtype=nev_dtype, mode='readwrite',
       offset=(16 * 2**10))

def mySpecgram(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    #print timeSlices
    for item in timeSlices:
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        if method == 'yule':
            p = sp.pyule(data, 550, norm='biased', NFFT=NFFT,sampling=Fs)
        elif method =='fft':
            p = sp.Periodogram(data,NFFT=NFFT,sampling=Fs)
        p();
        freqz = np.array(p.frequencies())
        Pxx.append(p.psd)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx


In [5]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/LinearTrack_12-22-15_Day2/01-sleep1/'
nvtFile = animalPath+'VT1.nvt'
localPath = '/mnt/Data/ephysdata/Rats/'
lfpPaths = []
for lfp in tools.locate('*-p*.lfp',animalPath):#animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)

In [6]:
lfp = pkl.load(open(lfpPaths[0],'rb'))

In [7]:
SleepPeriodsDF = pkl.load(open('sleep1Periods.pd','rb'))

In [8]:
swsIdx = SleepPeriodsDF[SleepPeriodsDF.epoch =='sws'].index
remIdx = SleepPeriodsDF[SleepPeriodsDF.epoch =='rem'].index

In [9]:
REMpsdStack = []
SWSpsdStack = []
REMpsdStackWhite = []
SWSpsdStackWhite = []
for ii in swsIdx:
    sampIdx = np.where(np.logical_and(lfp.timeAxis>SleepPeriodsDF.t0[ii],
                                  lfp.timeAxis<SleepPeriodsDF.t1[ii]))[0]
    flag = True
    while flag:
        sampSig = lfp.signal[sampIdx]
        p = sp.Periodogram(sampSig,sampling=2000,NFFT=2**15)
        try:
            p();
            flag = False
        except AssertionError:
            sampIdx = np.insert(sampIdx,-1,sampIdx[-1]+1)
    p.psd = MA(p.psd,10)
    freqz = np.power(np.array(p.frequencies()),2.5)
    SWSpsdStackWhite.append(np.multiply(p.psd,freqz))
    SWSpsdStack.append(p.psd)
for ii in remIdx:
    sampIdx = np.where(np.logical_and(lfp.timeAxis>SleepPeriodsDF.t0[ii],
                                  lfp.timeAxis<SleepPeriodsDF.t1[ii]))[0]
    flag = True
    while flag:
        sampSig = lfp.signal[sampIdx]
        p = sp.Periodogram(sampSig,sampling=2000,NFFT=2**15)
        try:
            p();
            flag = False
        except AssertionError:
            sampIdx = np.insert(sampIdx,-1,sampIdx[-1]+1)
    p.psd = MA(p.psd,10)
    freqz = np.power(np.array(p.frequencies()),2.5)
    REMpsdStackWhite.append(np.multiply(p.psd,freqz))
    REMpsdStack.append(p.psd)

/home/chenani/anaconda/lib/python2.7/site-packages/spectrum-0.6.1-py2.7-linux-x86_64.egg/spectrum/psd.py:451: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.__psd == None:


In [10]:
sns.tsplot(SWSpsdStack[:],time=p.frequencies())
sns.tsplot(SWSpsdStackWhite[:],time=p.frequencies(),c='c')
sns.tsplot(REMpsdStack[:],time=p.frequencies(),c='r')
sns.tsplot(REMpsdStackWhite[:],time=p.frequencies(),c='m')

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [11]:
pl.yscale('log')
pl.xscale('log')
pl.xlim(1,1e3)

(1, 1000.0)

In [59]:
p = sp.Periodogram(lfp.signal,NFFT=2**22,sampling=2e3)
p();
p.plot()

/home/chenani/anaconda/lib/python2.7/site-packages/spectrum-0.6.1-py2.7-linux-x86_64.egg/spectrum/psd.py:639: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.__psd == None:


In [61]:
freqz = np.array(p.frequencies())

In [124]:
pl.scatter(freqz,MA(p.psd,10))
pl.xscale('log')
pl.yscale('log')
pl.xlim(1,1000)
pl.ylim(1e-3,1e10)

(0.001, 10000000000.0)

In [63]:
p.get_

TypeError: get_converted_psd() takes exactly 2 arguments (1 given)

In [70]:
s = sp.Periodogram(lfp.signal[:1000])
s();

In [78]:
s.periodogram()

In [85]:
s.

False

In [34]:
lfp.estimateNoisefreqRelation(detectOutliers=False)

In [13]:
lfp.fft(display=True)

23


/home/chenani/anaconda/lib/python2.7/site-packages/spectrum-0.6.1-py2.7-linux-x86_64.egg/spectrum/psd.py:639: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.__psd == None:


In [41]:
lfp.estimateNoisefreqRelation(detectOutliers=True,noOfIterations = 50)

In [42]:
lfp.noiseFreqExponent

-2.440137742434362

In [43]:
lfp.flattenPowerspectrum()

In [56]:
pl.plot(lfp.freq[1:],MA(lfp.spPower,15))
pl.plot(lfp.freq[1:],MA(lfp.spPower_whitened,15)/30.0)

In [99]:
lfp.fft()

23


In [123]:
sampSig = lfp.signal[:2048]
print sampSig.size
ftrans = np.fft.rfft(sampSig,n=2048)
print ftrans.size
fback = np.fft.irfft(ftrans)
pl.plot(sampSig - fback)

2048
1025


In [137]:
lfp.whitenLinearRegression()

AttributeError: 'NeuralynxCSC' object has no attribute 'whitenLinearRegression'

In [138]:
lfp.filter(0,10)

In [139]:
signale.cscs.NeuralynxCSC.whitenLinearRegression

<unbound method NeuralynxCSC.whitenLinearRegression>

In [121]:
np.fft.irfft(lfp.sp).size

7127552